##  Demonstration of Grapheme enabled sub-tokenization

In [1]:
#!pip install gdown

import gdown
import numpy as np
import os

### Let's load the text file (Tamil)

In [2]:
# let's down the Tamil example text file from gdrive
PATH = "./tamil.txt"
if not os.path.isfile("./tamil.txt"):
    path = gdown.download(url="https://drive.google.com/file/d/1-065EvxQepsQWTpDbdAfgPw0kkiwfMcR/view?usp=drive_link", output=PATH, fuzzy=True)
    PATH = path

LANG = "ta"

In [3]:
# load up the text file.
with open(PATH, "r") as file:
    text = file.read()
    file.close()
    size = len(text)

In [4]:
# view a small piece of the text document.
LEN = 200
pos = np.random.randint(0, size-LEN)
print(text[pos:pos+LEN])

்கச் செய்தி ஊடகத்தால் கூறப்படுவது போல் நிராகரிக்கப்பட்டு விட்டால் , அதன் விளைவு தவிர்க்க முடியாத வெள்ளைப் பூச்சாகத்தன் இருக்கும்; அப்படித்தான் 911 தாக்குதல்கள் பற்றி பல உத்தியோகபூர்வ விசாரணைகளின் கத


### Let's encode the complex unicode rendering into singular unicode per grapheme

In [5]:
# Indic Unicode Mapper maps the sequence of unicode that constitute a grapheme 
# into a singular unicode in the 0xE00X range.
from indic_unicode_mapper import IndicUnicodeMapper
mapper = IndicUnicodeMapper()

In [6]:
# encode the graphemes.
encoded_text = mapper.encode(text=text, lang=LANG)

In [7]:
# the encode text will have unicodes in the 0xE0XX range.
pos = np.random.randint(0, len(encoded_text)-LEN)
encoded_text[pos:pos+LEN]

'எ\ue0a3\ue136\ue15b\ue0b2\ue0a5\ue181ளன\ue13c; \ue104\ue15d\ue10e அ\ue103\ue12d\ue011\ue001 \ue101\ue1cdர\ue0f7\ue0eaட\ue0e0 \ue0be\ue12f\ue011க\ue0feன ந\ue084\ue0ee\ue0b2 \ue0acடர \ue1a5\ue09b\ue077\ue10e எ\ue0e0\ue146\ue10e , எ\ue16a\ue15aவ\ue153\ue144\ue004\ue10e \ue104\ue15aக அவ\ue12f\ue07bய ஆ\ue084\ue030 ந\ue10e\ue0e8\ue116\ue12f\ue011\ue004\ue10e \ue0e7\ue011\ue002\ue1f4\ue0a2\ue0d1ய இ\ue12c\ue08eவ\ue0b2\ue0a5ட\ue0d3\ue10e \ue0be\ue12f\ue011க\ue0feக இ\ue12f\ue011க \ue1a5\ue09b\ue077\ue10e எ\ue0e0\ue146\ue10e \ue005\ue144\ue118\ue181ளன\ue13c .\nஎ\ue0e0\ue0d7 \ue0b9\ue0d6 இ\ue028\ue004 அ\ue18f\ue0b2\ue0a5 வ\ue0c9\ue0a5\ue181\ue177\ue0e0 ! .\nபட\ue0b2\ue0a5\ue011\ue001ன ட\ue0f7\ue0e8\ue028\ue004\ue10e ர\ue047\ue0d1\ue11b \ue0ed\ue032\ue19f\ue13c என த\ue115\ue12d\ue0f7\ue0e7'

### Let's now learn a BERT Tokenizer model on the encoded dataset.

In [8]:
# instantiate the BERT WPE tokenizer module.
from tokenizers import BertWordPieceTokenizer

cls_token = "[cls]"
sep_token = "[sep]"
mask_token = "[mask]"
pad_token = "[pad]"
unk_token = "[unk]"
spl_tokens = ["[unk]", "[sep]", "[mask]", "[cls]", "[pad]"]  # special tokens
tokenizer = BertWordPieceTokenizer(clean_text=False, 
                                   handle_chinese_chars=True, 
                                   strip_accents=False,
                                   lowercase=False,
                                   sep_token=sep_token, unk_token=unk_token, 
                                   mask_token=mask_token, cls_token=cls_token, pad_token=pad_token)

# setup the Vocabulary size requirement
VOCAB_SIZE = 3000

In [9]:
# we need a temporary folder to keep the encoded file(s) there.
tmpdir = f"/tmp/mapped-{LANG}"
os.makedirs(tmpdir, exist_ok=True)

with open(tmpdir + "/" + PATH, "w") as ofile:
    ofile.write(encoded_text)
    ofile.close()

# add our encoded text file to the array of paths.
files = [tmpdir + "/" + PATH]

In [10]:
# train the algorithm
tokenizer.train(files=files, vocab_size=VOCAB_SIZE, min_frequency=2,
                limit_alphabet=512, wordpieces_prefix='##',
                special_tokens=spl_tokens)

In [11]:
# save the tokenization model.
# this line should create a file with the name f"{LANG}-vocab.txt"
tokenizer.save_model('.', LANG)
TOKENIZER_MODEL = f"{LANG}-vocab.txt"

#### If you open the text file, you will see a lot of gibberish as the words are in encoded form.  Let's display a human understandable version side-by-side.

In [12]:
# let's load from the model file.
with open(TOKENIZER_MODEL, "r") as vfile:
    lines = vfile.readlines()
    vfile.close()

In [13]:
nlines = len(lines)
print(f"found {nlines} vocab items.")
# we are drawing the lines from the latter half, 
# as the tokens are longer there.
lpos = np.random.randint(0, nlines-5)

print("Encoded Strings".ljust(20,' '), "\tOriginal Strings".ljust(20, ' '))
print("".ljust(20,'='), "\t".ljust(20, '='))
for l in range(lpos, lpos+10):
    print("%s\t%s" % (lines[l].rstrip().ljust(20,' '), mapper.decode(lines[l]).rstrip().ljust(10, ' ')))

found 3000 vocab items.
Encoded Strings      	Original Strings   
==================== 	===================
வறன              	வருகின்றன 
அவ               	அனைவரும்  
ர                	குமாரன்   
தகப                	தகப்ப     
##கபள          	##க்கப்பட்டுள்ள
கசன               	கணிசமான   
ர                 	முரண்     
டப              	செப்டம்பர்
ஆ                 	ஆய்வு     
தள                 	தள்ள      


### Now, let's tokenize the text using the grapheme enabled WPE tokenizer.

In [14]:
# load the tokenizer model that inherently does the unicode encoding/decoding.
from indic_bert_tokenizer import IndicBertWordPieceTokenizer
tokenizer = IndicBertWordPieceTokenizer(model_path=TOKENIZER_MODEL)

In [15]:
# just use a random text here.
test_text = "ஆனால் மொத்த ஊக்கப் பொதியின் சிறிய அளவு வரி வெட்டுக்கள் மற்றும் செலவின அதிகரிப்புக்கள் இணைந்தது அரசாங்கத்தின் பிற்போக்குத்தன்மையை காட்டும் நடவடிக்கை அல்ல"

In [16]:
# encode the input text into token encodings.
toks = tokenizer.encode(test_text)
toks

Encoding(num_tokens=33, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [17]:
# list to token ids.
print(toks.ids)

[3, 889, 2453, 49, 672, 365, 214, 389, 739, 1969, 2404, 1945, 2925, 791, 711, 1394, 383, 408, 1403, 1488, 1233, 795, 1639, 206, 2174, 668, 2631, 439, 1624, 414, 901, 888, 1]


In [18]:
# list the token string (in encoded format)
print(toks.tokens)

['[cls]', 'ஆ\ue0d0\ue16a', '\ue108\ue0b2த', 'ஊ', '##\ue011க', '##\ue0f7', '\ue0f1', '##\ue0a3', '##\ue116\ue0e0', '\ue030\ue144ய', 'அள\ue1a2', 'வ\ue12d', '\ue1a4\ue084\ue077', '##\ue011க\ue181', 'ம\ue153\ue146\ue10e', '\ue034ல', '##\ue1a0', '##ன', 'அ\ue0a3க\ue12d', '##\ue0f7\ue0ea\ue011க\ue181', 'இ\ue092', '##\ue0c9த\ue0a5', 'அர\ue02f\ue028க\ue0b2\ue0a3\ue0e0', '\ue0e8', '##\ue153\ue0f9', '##\ue011\ue004', '##\ue0b2த\ue0e0\ue105', '##\ue11c', '\ue001\ue084\ue077', '##\ue10e', 'நடவ\ue075\ue011\ue008', 'அ\ue16aல', '[sep]']


In [19]:
print([tokenizer.decode_string(tok) for tok in toks.tokens])


['[cls]', 'ஆனால்', 'மொத்த', 'ஊ', '##க்க', '##ப்', 'பொ', '##தி', '##யின்', 'சிறிய', 'அளவு', 'வரி', 'வெட்டு', '##க்கள்', 'மற்றும்', 'செல', '##வி', '##ன', 'அதிகரி', '##ப்புக்கள்', 'இணை', '##ந்தது', 'அரசாங்கத்தின்', 'பி', '##ற்போ', '##க்கு', '##த்தன்மை', '##யை', 'காட்டு', '##ம்', 'நடவடிக்கை', 'அல்ல', '[sep]']


In [20]:
# get the decoded string from the tokenizer ids.
tokenizer.decode(toks.ids)

'ஆனால் மொத்த ஊக்கப் பொதியின் சிறிய அளவு வரி வெட்டுக்கள் மற்றும் செலவின அதிகரிப்புக்கள் இணைந்தது அரசாங்கத்தின் பிற்போக்குத்தன்மையை காட்டும் நடவடிக்கை அல்ல'

In [21]:
# also check if the input and the decoded string are matching
assert(test_text == tokenizer.decode(toks.ids))